# Maxflow problem

A network is represent in the diagram below:

![alt text](https://i.imgur.com/XYDSpjY.png)

The number indicates the capacity of the flow in each direction. For example, the flow capacity from A to G is 10 units, but the flow capacity from G to A is 0.

Find the maximum flow of this network where A is the source and J is the sink.